# Proof of concept for running qml with pennylane with noisy devices and hw

In [1]:
import pennylane as qml
import torch
from torch.autograd import Variable

In [2]:
def cost(phi, theta, step):
    target = -(-1) ** (step // 100)
    return torch.abs(circuit(phi, theta) - target) ** 2

## Running default

In [3]:
# construct the device
dev = qml.device("default.qubit")

In [4]:
@qml.qnode(dev, interface="torch")
def circuit(phi, theta):
    qml.RX(theta, wires=0)
    qml.RZ(phi, wires=0)
    return qml.expval(qml.PauliZ(0))

In [5]:
phi = Variable(torch.tensor(1.0), requires_grad=True)
theta = Variable(torch.tensor(0.05), requires_grad=True)
opt = torch.optim.Adam([phi, theta], lr=0.1)

In [6]:
for i in range(400):
    opt.zero_grad()
    loss = cost(phi, theta, i)
    loss.backward()
    opt.step()

In [7]:
print(phi)
print(theta)
print(circuit(phi, theta))
print(cost(phi, theta, 400))

tensor(2.9222, requires_grad=True)
tensor(12.7774, requires_grad=True)
tensor(0.9778, dtype=torch.float64, grad_fn=<DotBackward0>)
tensor(3.9118, dtype=torch.float64, grad_fn=<PowBackward0>)


## Running with noise

In [8]:
# construct the device
dev = qml.device("default.mixed", wires=[0])

In [9]:
@qml.qnode(dev, interface="torch")
def circuit(phi, theta):
    qml.RX(theta, wires=0)
    qml.RZ(phi, wires=0)
    qml.DepolarizingChannel(0.01, wires=[0])
    return qml.expval(qml.PauliZ(0))

In [10]:
phi = Variable(torch.tensor(1.0), requires_grad=True)
theta = Variable(torch.tensor(0.05), requires_grad=True)
opt = torch.optim.Adam([phi, theta], lr=0.1)

In [11]:
for i in range(400):
    opt.zero_grad()
    loss = cost(phi, theta, i)
    loss.backward()
    opt.step()

In [12]:
print(phi)
print(theta)
print(circuit(phi, theta))
print(cost(phi, theta, 400))

tensor(3.1045, requires_grad=True)
tensor(12.7501, requires_grad=True)
tensor(0.9701, dtype=torch.float64, grad_fn=<DotBackward0>)
tensor(3.8811, dtype=torch.float64, grad_fn=<PowBackward0>)


## Runnin in qiskit fake-device

In [13]:
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke

fake_backend = FakeSherbrooke()

# passing a string in backend would result in an error
dev = qml.device('qiskit.remote', wires=127, backend=fake_backend)

In [14]:
@qml.qnode(dev, interface="torch")
def circuit(phi, theta):
    qml.RX(theta, wires=0)
    qml.RZ(phi, wires=0)
    #qml.DepolarizingChannel(0.01, wires=[0])
    return qml.expval(qml.PauliZ(0))

In [15]:
phi = Variable(torch.tensor(1.0), requires_grad=True)
theta = Variable(torch.tensor(0.05), requires_grad=True)
opt = torch.optim.Adam([phi, theta], lr=0.1)

In [17]:
for i in range(400):
    if i%10 == 0:
        print(i)
    opt.zero_grad()
    loss = cost(phi, theta, i)
    loss.backward()
    opt.step()

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390


In [18]:
print(phi)
print(theta)
print(circuit(phi, theta))
print(cost(phi, theta, 400))

tensor(0.2868, requires_grad=True)
tensor(1.6978, requires_grad=True)
tensor(0.9941, dtype=torch.float64, grad_fn=<ExecuteTapesBackward>)
tensor(3.9766, dtype=torch.float64, grad_fn=<PowBackward0>)


## Running in hw (obsolete because of runtime)

In [31]:
from qiskit_ibm_runtime import QiskitRuntimeService

#QiskitRuntimeService.save_account(channel="ibm_quantum", token="")
# To access saved credentials for the IBM quantum channel and select an instance
service = QiskitRuntimeService(channel="ibm_quantum", instance="ibm-q/open/main")
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=1)

# passing a string in backend would result in an error
dev = qml.device('qiskit.remote', wires=127, backend=backend)

In [32]:
@qml.qnode(dev, interface="torch")
def circuit(phi, theta):
    qml.RX(theta, wires=0)
    qml.RZ(phi, wires=0)
    #qml.DepolarizingChannel(0.01, wires=[0])
    return qml.expval(qml.PauliZ(0))

In [33]:
phi = Variable(torch.tensor(1.0), requires_grad=True)
theta = Variable(torch.tensor(0.05), requires_grad=True)
opt = torch.optim.Adam([phi, theta], lr=0.1)

In [34]:
for i in range(400):
    opt.zero_grad()
    loss = cost(phi, theta, i)
    loss.backward()
    opt.step()

IBMRuntimeError: 'Failed to run program: \'403 Client Error: Forbidden for url: https://api.quantum.ibm.com/runtime/jobs. {"errors":[{"message":"Job create exceeds open plan job usage limits","code":4317,"solution":"Please wait until the beginning of next month to submit more jobs when your quota will reset.","more_info":"https://docs.quantum-computing.ibm.com/errors"}]}\''

In [35]:
print(phi)
print(theta)
print(circuit(phi, theta))
print(cost(phi, theta, 400))

tensor(0.7735, requires_grad=True)
tensor(0.2295, requires_grad=True)


IBMRuntimeError: 'Failed to run program: \'403 Client Error: Forbidden for url: https://api.quantum.ibm.com/runtime/jobs. {"errors":[{"message":"Job create exceeds open plan job usage limits","code":4317,"solution":"Please wait until the beginning of next month to submit more jobs when your quota will reset.","more_info":"https://docs.quantum-computing.ibm.com/errors"}]}\''